<a href="https://colab.research.google.com/github/calvinli08/tf-demo/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras import layers

import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

#get dataset and format
(train_ds, validate_ds, test_ds), ds_info = tfds.load('HorsesOrHumans', split=['train[:90%]', 'train[90%:100%]', 'test'], as_supervised=True, shuffle_files=True, with_info=True, batch_size=32)

#normalize data so the model trains more efficiently
def normalize(image, label):
  image = tf.cast(image/255, tf.float32)
  return image, label

train_ds = train_ds.map(normalize)
validate_ds = validate_ds.map(normalize)
test_ds = test_ds.map(normalize)

In [ ]:
#prepare model layers
model = keras.Sequential([
    #randomize the training data to increase data diversity
    layers.RandomFlip("horizontal_and_vertical"),

    #use convolution to highlight the parts of the image most useful for identification
    #kernel regularizer should help prevent overfitting
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3), kernel_regularizer=regularizers.l2(0.01)),

    #cut down on excess image info returned from the convolution layer and keep just the essentials
    layers.MaxPooling2D(2,2),

    #second convolution layer
    layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    #cut down on excess image info returned from the convolution layer and keep just the essentials
    layers.MaxPooling2D(2,2),

    #third convolution layer
    layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    #cut down on excess image info returned from the convolution layer and keep just the essentials
    layers.MaxPooling2D(2,2),

    #prepare to feed to dense layer
    layers.Flatten(),

    #classify using dense layers
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    layers.Dropout(0.5),

    layers.Dense(2)
])

#set model optimizer and loss equation
model.compile(
    optimizer='adam', #Using adam since it's quickest and most accurate for basic data sets
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #Since we have two categories, human and horse, this should be most effective
    metrics=[
      'accuracy'
    ]
)

model.build((None,300,300,3))

model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_15 (RandomFlip  (None, 300, 300, 3)       0         
 )                                                               
                                                                 
 conv2d_33 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 149, 149, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_34 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_34 (MaxPooli  (None, 73, 73, 32)        0         
 ng2D)                                                           
                                                     

In [ ]:
#train and test (using just 5 epochs to save resources as it's just a demo)
checkpoint_filepath = 'quick_demo.cpkt'
save_cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, verbose=1)

early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=200)

model.fit(
    train_ds,
    epochs=5,
    validation_data=validate_ds,
    batch_size=32,
    callbacks=[
        save_cp_callback,
        early_stopping_callback
    ]
)

loss, acc = model.evaluate(test_ds, verbose=2)

print("\nTest results - Loss:", "{:.2f}".format(loss), "Accuracy:", "{:.2f}".format(acc))

Epoch 1/5
29/29 [==============================] - ETA: 0s - loss: 1.5066 - accuracy: 0.6732
Epoch 1: saving model to quick_demo.cpkt
29/29 [==============================] - 7s 185ms/step - loss: 1.5066 - accuracy: 0.6732 - val_loss: 0.8120 - val_accuracy: 0.9029
Epoch 2/5
29/29 [==============================] - ETA: 0s - loss: 0.7205 - accuracy: 0.9113
Epoch 2: saving model to quick_demo.cpkt
29/29 [==============================] - 4s 140ms/step - loss: 0.7205 - accuracy: 0.9113 - val_loss: 0.6390 - val_accuracy: 0.8835
Epoch 3/5
29/29 [==============================] - ETA: 0s - loss: 0.6049 - accuracy: 0.9253
Epoch 3: saving model to quick_demo.cpkt
29/29 [==============================] - 4s 137ms/step - loss: 0.6049 - accuracy: 0.9253 - val_loss: 0.5394 - val_accuracy: 0.9126
Epoch 4/5
29/29 [==============================] - ETA: 0s - loss: 0.5123 - accuracy: 0.9340
Epoch 4: saving model to quick_demo.cpkt
29/29 [==============================] - 5s 183ms/step - loss: 0.5123 -